# RF6 Impact Attestation Metrics

Spec [here](https://plaid-cement-e44.notion.site/Impact-Attestations-Data-b8b1c79a624c46ad94305c74def64783).

In [1]:
import datetime
from dotenv import load_dotenv
import json
import numpy as np
import os
import pandas as pd
import requests

# Get Farcaster

In [2]:
from google.cloud import bigquery

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../../../gcp_credentials.json'
client = bigquery.Client()
PROJECT = 'opensource-observer'
query = f"""
WITH
  profiles AS (
    SELECT
      v.fid,
      v.address,
      p.custody_address
    FROM `{PROJECT}.farcaster.verifications` v
    JOIN `{PROJECT}.farcaster.profiles` p ON v.fid = p.fid
    WHERE v.deleted_at IS NULL
  ),
  eth_addresses AS (
    SELECT
      fid,
      address
    FROM profiles
    WHERE LENGTH(address) = 42
    UNION ALL
    SELECT
      fid,
      custody_address AS address
    FROM profiles
  )
SELECT DISTINCT
  fid,
  address
FROM eth_addresses
"""

result = client.query(query)
farcaster = result.to_dataframe()
FARCASTER = farcaster.groupby('fid')['address'].agg(set).to_dict()
farcaster.tail(3)

,fid,address
735261,524287,0xebb5279b293ac897c311c7560fe48163a9ddd0d1
735262,589823,0xfd7063944f06613b71141c4d25d068a52e65a882
735263,720895,0xc68cfe5517be77f4253ca01385c00ac216811a6e


# Get delegate info

In [3]:
load_dotenv()
AGORA_API_KEY = os.environ['AGORA_API_KEY']

url = 'https://vote.optimism.io/api/v1/delegates'
params = {'limit': 100, 'offset': 0, 'sort': 'most_delegators'}
headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {AGORA_API_KEY}'
}
response = requests.get(url, params=params, headers=headers)
if response.status_code == 200:
    agora_data = response.json()
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)

In [4]:
DELEGATES = [x['address'].lower() for x in agora_data['data']]
len(DELEGATES)
# for p in DELEGATES:
#     print(f"'{p}',")

100

# Load relevant attestations

In [5]:
def load_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data

In [6]:
project_attestation_data = load_json('data/attestations/472.json')
print("Project attestations:", len(project_attestation_data))

app_attestation_data = load_json('data/attestations/609.json')
print("Application attestations:", len(app_attestation_data))

badgeholder_attestation_data = load_json('data/attestations/599.json')
print("Badgeholder attestations:", len(badgeholder_attestation_data))

gov_council_attestation_data = load_json('data/attestations/141.json')
print("Gov Council attestations:", len(gov_council_attestation_data))

impact_attestation_data = load_json('data/attestations/566.json')
print("Impact attestations:", len(impact_attestation_data))

Project attestations: 4234
Application attestations: 396
Badgeholder attestations: 304
Gov Council attestations: 217
Impact attestations: 371


In [7]:
ATTESTER_ADDR = "0xf6872d315cc2e1aff6abae5dd814fd54755fe97c"

df_project_metadata = pd.DataFrame(project_attestation_data)
df_project_metadata = df_project_metadata[df_project_metadata.attester.str.lower() == ATTESTER_ADDR]
df_project_metadata.sort_values(by="timeCreated", ascending=False, inplace=True)
df_project_metadata.drop_duplicates(subset="projectRefUID", keep="first", inplace=True)

PROJECTS = list(df_project_metadata['projectRefUID'].unique())
print("Projects:", len(PROJECTS))

df_project_metadata.iloc[0]

Projects: 3020


id                     0x36e632d6651278f4ed89f3eccbf0e7a5245741a4da89...
attester                      0xF6872D315CC2E1AfF6abae5dd814fd54755fE97C
recipient                     0x0000000000000000000000000000000000000000
timeCreated                                                   1729066136
projectRefUID          0x25d3cb05bc73d81fb4c84bac183594cbacf6e140a98c...
farcasterID                     {'type': 'BigNumber', 'hex': '0x0758c4'}
name                                                        boribori.eth
category                                                          Social
parentProjectRefUID    0x00000000000000000000000000000000000000000000...
metadataType                                                           0
metadataUrl            https://storage.retrofunding.optimism.io/ipfs/...
metadata               {'name': 'boribori.eth', 'description': 'Farca...
Name: 4231, dtype: object

In [8]:
df_applications = pd.DataFrame(app_attestation_data)
df_applications = df_applications[df_applications.attester.str.lower() == ATTESTER_ADDR]
df_applications['round'] = df_applications['round'].apply(lambda x: str(x))
df_applications.sort_values(by="timeCreated", ascending=False, inplace=True)
df_applications.drop_duplicates(subset="metadataSnapshotRefUID", keep="first", inplace=True)

latest_metadata = list(df_project_metadata.id.unique())
df_applications = df_applications[df_applications.metadataSnapshotRefUID.isin(latest_metadata)]

print("Applications:", len(df_applications))
df_applications.iloc[0]

Applications: 135


id                        0x336ac39aefc89a9f2bb8832f362074d32a95a9d6fabe...
attester                         0xF6872D315CC2E1AfF6abae5dd814fd54755fE97C
recipient                        0x0000000000000000000000000000000000000000
timeCreated                                                      1728932322
round                                                                     6
farcasterID                        {'type': 'BigNumber', 'hex': '0x033bdf'}
metadataSnapshotRefUID    0xf17ee19c8307170ca4d77c571580e8ad2f304727360b...
metadataType                                                              0
metadataUrl               https://storage.retrofunding.optimism.io/ipfs/...
metadata                  {'round': 6, 'category': 'Governance Infra & T...
Name: 165, dtype: object

# Clean / filter attestation data

In [9]:
relevant_badgeholder_attestations = []
for a in badgeholder_attestation_data:
    if a['attester'] != '0xE4553b743E74dA3424Ac51f8C1E586fd43aE226F':
        continue
    if a['voterType'] == 'Guest':
        continue
    relevant_badgeholder_attestations.append({
        'id': a['id'].lower(),
        'attester': a['attester'].lower(),
        'recipient': a['recipient'].lower(),
        'timeCreated': datetime.datetime.utcfromtimestamp(a['timeCreated'])
    })
print("Relevant badgeholder attestations:", len(relevant_badgeholder_attestations))            

Relevant badgeholder attestations: 200


In [10]:
mapped_roles = {
    'ACC (Second half)': 'Anticapture Commission',
    'Anticapture Commission Member': 'Anticapture Commission',
    'Badgeholder Reviewer': None,
    'Code of Conduct Council': 'Code of Conduct',
    'Code of Conduct Council Lead': 'Code of Conduct',
    'Code of Conduct Council Member': 'Code of Conduct',
    'Developer Advisory Board Lead': 'Developer Advisory Board',
    'Developer Advisory Board Member': 'Developer Advisory Board',
    'Optimism Grants Council Lead': 'Grants Council',
    'Optimism Grants Council Member': 'Grants Council',
    'Security Council Lead': 'Security Council',
    'Security Council Member': 'Security Council',
}
relevant_gov_council_attestations = []
for a in gov_council_attestation_data:
    if a['attester'] != '0xE4553b743E74dA3424Ac51f8C1E586fd43aE226F':
        continue
    if a['govSeason'] not in ['5', '6']:
        continue
    gov_role = mapped_roles.get(a['govRole'], 'unknown')
    if not gov_role:
        continue
    if gov_role == 'unknown':
        print("Unknown role:", a['govRole'])
        continue
    relevant_gov_council_attestations.append({
        'id': a['id'].lower(),
        'attester': a['attester'].lower(),
        'recipient': a['recipient'].lower(),
        'govRole': gov_role,
        'timeCreated': datetime.datetime.utcfromtimestamp(a['timeCreated'])
    })
print("Relevant gov council attestations:", len(relevant_gov_council_attestations))            

Relevant gov council attestations: 60


# Determine approved attestation recipients

In [11]:
COUNCILS = list(pd.DataFrame(relevant_gov_council_attestations)['govRole'].unique())
COUNCILS

['Grants Council',
 'Anticapture Commission',
 'Developer Advisory Board',
 'Code of Conduct',
 'Security Council']

In [12]:
BADGEHOLDERS = set([a['recipient'].lower() for a in relevant_badgeholder_attestations])
len(BADGEHOLDERS)

103

In [13]:
ATTESTERS = {}
for addr in BADGEHOLDERS:
    if addr not in ATTESTERS:
        ATTESTERS.update({
            addr: {
                'is_citizen': True,
                'is_top_delegate': False,
                'governance_membership': set()                
            }
        })
for addr in DELEGATES:
    if addr not in ATTESTERS:
        ATTESTERS.update({
            addr: {
                'is_citizen': False,
                'is_top_delegate': True,
                'governance_membership': set()
            }
        })
    else:
        ATTESTERS[addr]['is_top_delegate'] = True

for a in relevant_gov_council_attestations:
    addr = a['recipient'].lower()
    if addr in ATTESTERS:
        ATTESTERS[addr]['governance_membership'].add(a['govRole'])  
        
for addr in ATTESTERS:
    ATTESTERS[addr]['governance_membership'] = list(ATTESTERS[addr]['governance_membership'])    

print(len(ATTESTERS.keys()))

188


# Create attestations dataframe

In [14]:
def convert_farcaster(farcaster_id):
    return int(farcaster_id['hex'], 16) if farcaster_id else None

In [15]:
mgl_attester = '0x7484aABFef9f39464F332e632047983b67571C0a'

reviewed_attestations = []
relevant_impact_attestations = []
for attestation in impact_attestation_data:

    a = attestation.copy()
    fid = convert_farcaster(a['farcasterID'])
    a['farcasterID'] = fid
    a['timeCreated'] = datetime.datetime.utcfromtimestamp(a['timeCreated'])
    
    if a['attester'] != mgl_attester:
        a.update({'status': 'unknown_signer'})
        reviewed_attestations.append(a)
        continue
        
    if a['projectRegUID'] not in PROJECTS:
        a.update({'status': 'unknown_project'})
        reviewed_attestations.append(a)
        continue

    user_addresses = FARCASTER.get(fid, set())
    if not len(user_addresses):
        a.update({'status': 'farcaster_has_no_eoa'})
        reviewed_attestations.append(a)
        continue
    
    linked_addresses = user_addresses.intersection(set(ATTESTERS.keys()))
    if not linked_addresses:
        a.update({'status': 'farcaster_not_gov_participant'})
        reviewed_attestations.append(a)
        continue

    if len(linked_addresses) > 1:
        print(fid)
    linked_address = linked_addresses.pop()
        
    qs = a.get('metadata', {}).get('impactAttestations', [])
    if len(qs) < 2:
        a.update({'status': 'missing_metadata'})
        reviewed_attestations.append(a)
        continue
    
    nps = pmf = None
    for q in qs:
        val = q.get('value')
        if not val:
            continue
        if q['name'] == 'Likely to Recommend':
            nps = val
        elif q['name'] == 'Feeling if didnt exist':
            pmf = val
    
    if not (nps or pmf):
        a.update({'status': 'missing_scores'})
        reviewed_attestations.append(a)
        continue

    a.update({'status': 'OK'})
    reviewed_attestations.append(a)
    relevant_impact_attestations.append({
        'id': a['id'].lower(),
        'attester': a['attester'].lower(),
        'recipient': a['recipient'].lower(),
        'linkedAddress': linked_address,
        'farcasterID': fid,
        'projectRegUID': a['projectRegUID'].lower(),
        'timeCreated': a['timeCreated'],
        'issuer': a['issuer'],
        'metadataurl': a['metadataurl'],
        'nps_score': pd.to_numeric(nps),
        'pmf_score': pd.to_numeric(pmf)
    })
    
print("Reviewed impact attestations:", len(reviewed_attestations))
print("Relevant impact attestations:", len(relevant_impact_attestations))

Reviewed impact attestations: 371
Relevant impact attestations: 68


In [16]:
df_attestations_actual = pd.DataFrame(relevant_impact_attestations)
df_attestations_actual.tail(1).T

,67
id,0xe7b8fbea065ab151b55bf4a84cc06ccb3909a46007d1...
attester,0x7484aabfef9f39464f332e632047983b67571c0a
recipient,0x9934465ee73beaf148b1b3ff232c8cd86c4c2c63
linkedAddress,0x64fed9e56b548343e7bb47c49ecd7ffa9f1a34fe
farcasterID,777864
projectRegUID,0xe2247097961708233325f253d676ba2f9d8011147e1b...
timeCreated,2024-10-18 16:43:39
issuer,MGL
metadataurl,https://gateway.pinata.cloud/ipfs/QmW5U7WHVmXs...
nps_score,6.0


In [17]:
df_reviewed = pd.DataFrame(reviewed_attestations)
df_reviewed[df_reviewed['status'] == 'farcaster_not_gov_participant']['farcasterID'].value_counts()

farcasterID
484399    25
191212    15
21291     11
4469       8
18344      4
429828     4
20605      2
6187       2
14496      2
13462      1
18138      1
320694     1
260400     1
325302     1
189380     1
838        1
295767     1
217355     1
2588       1
Name: count, dtype: int64

# Implement a few of the metrics

In [18]:
df = df_attestations_actual.copy()
df['is_citizen'] = df.linkedAddress.apply(lambda x: ATTESTERS[x]['is_citizen'])
df['is_top_delegate'] = df.linkedAddress.apply(lambda x: ATTESTERS[x]['is_top_delegate'])
df['governance_membership'] = df.linkedAddress.apply(lambda x: ATTESTERS[x]['governance_membership'])
len(df)

68

In [19]:
def most_positive_superlative(ref_uid):
    dff = df[df['projectRegUID'] == ref_uid]
    num_reviews = len(dff)
    pmf_pos_reviews = (dff['pmf_score'] > 2).sum() / num_reviews
    nps_pos_reviews = (dff['nps_score'] > 8).sum() / num_reviews
    
    result = (num_reviews >= 20) and (pmf_pos_reviews >= .95) and (nps_pos_reviews >= .95)
    
    return str(result)

def cant_live_without_superlative(ref_uid):
    dff = df[df['projectRegUID'] == ref_uid]
    num_reviews = len(dff)
    pmf_pos_reviews = (dff['pmf_score'] > 2).sum() / num_reviews
    
    result = (num_reviews >= 20) and (pmf_pos_reviews >= .90)
    
    return str(result)

def ratings_distribution(ref_uid):
    dff = df[df['projectRegUID'] == ref_uid]
    dff_citizens = dff[dff.is_citizen == True]
    dff_delegates = dff[dff.is_top_delegate == True]

    def calculate_distribution(group, score):
        total = len(group)
        if not total:
            return None
        return (group['pmf_score'] == score).sum() / total
    
    return {
        'citizens': {
            'extremely_upset': calculate_distribution(dff_citizens, 3),
            'somewhat_upset': calculate_distribution(dff_citizens, 2),
            'neutral': calculate_distribution(dff_citizens, 1)
        },
        'top_delegates': {
            'extremely_upset': calculate_distribution(dff_delegates, 3),
            'somewhat_upset': calculate_distribution(dff_delegates, 2),
            'neutral': calculate_distribution(dff_delegates, 1)
        }
    }

def councils_distribution(ref_uid):
    dff = df[df['projectRegUID'] == ref_uid]
    result = {}
    for council in COUNCILS:
        dff_council = dff[dff['governance_membership'].apply(lambda x: council in x)]
        if dff_council.empty:
            continue
        result.update({
            council.replace(' ','_').lower(): {
                'count_attestations': len(dff_council),
                'avg_pmf_score': dff_council['pmf_score'].mean(),
                'avg_nps_score': dff_council['nps_score'].mean()
            }
        })
    return result

In [20]:
metrics = []
metrics.append({
    'name': 'count_total_attestations',
    'description': 'Number of Attestations',
    'data': df.groupby('projectRegUID')['id'].count().to_dict()
})
metrics.append({
    'name': 'count_citizen_attestations',
    'description': 'Number of Attestations by Citizens',
    'data': df[df.is_citizen].groupby('projectRegUID')['id'].count().to_dict()
})
metrics.append({
    'name': 'count_delegate_attestations',
    'description': 'Number of Attestations by Top Delegates',
    'data': df[df.is_top_delegate].groupby('projectRegUID')['id'].count().to_dict()    
})
metrics.append({
    'name': 'avg_nps_score',
    'description': 'Average NPS score of Citizens and Top Delegates',
    'data': df.groupby('projectRegUID')['nps_score'].mean().to_dict()
})
metrics.append({
    'name': 'most_positive_superlative',
    'description': 'Most positive superlative (20 reviews, 95% high PMF and 95% high NPS)',
    'data': {uid: most_positive_superlative(uid) for uid in df['projectRegUID'].unique()}
})
metrics.append({
    'name': 'cant_live_without_superlative',
    'description': "Can't live without superlative (20 reviews, 90% high PMF)",
    'data': {uid: cant_live_without_superlative(uid) for uid in df['projectRegUID'].unique()}
})
metrics.append({
    'name': 'percentage_distributions',
    'description': "Percentage distribution of different ratings by citizens and top delegates",
    'data': {uid: ratings_distribution(uid) for uid in df['projectRegUID'].unique()}
})    
metrics.append({
    'name': 'elected_governance_reviews',
    'description': 'Reviews from elected governance members',
    'data': {uid: councils_distribution(uid) for uid in df['projectRegUID'].unique()}
})

In [21]:
with open("data/attestation_metrics.json", "w") as f:
    json.dump(metrics,f,indent=2)

In [22]:
pd.DataFrame(reviewed_attestations).to_csv("data/normalized_attestations.csv")